In [1]:
#### Convert coffea output into root TH2's to feed into TUnfold
import coffea
import pickle
import uproot
import awkward as ak
import matplotlib.pyplot as plt
import numpy as np
import os
# import ROOT

%load_ext autoreload

%autoreload 2

In [17]:
fname = "coffeaOutput/dijet/dijetHists_deltaRmatch0p1_QCDsim_rap2.5_herwig_nominal__2018.pkl"
with open(fname, "rb") as f:
    result = pickle.load( f )
print(result['ptreco_mreco_g'][{'syst':'nominal'}])

fname = "coffeaOutput/trijet/trijetHists_removeJMRJMS_QCDsim_rap2.5_herwig_allUncALLRespOnly.pkl"
with open(fname, "rb") as f:
    result = pickle.load( f )
print(result['misses_u'][{'syst':'nominal'}])

Hist(
  StrCategory(['herwig2018'], growth=True, name='dataset', label='Primary dataset'),
  IntCategory([-1], growth=True, name='jk', label='Jackknife section'),
  Variable([200, 290, 400, 480, 570, 680, 760, 13000], name='ptreco', label='p_{T,RECO} (GeV)'),
  Variable(array([   0. ,    2.5,    5. ,    7.5,   10. ,   15. ,   20. ,   30. ,
         40. ,   50. ,   60. ,   70. ,   80. ,   90. ,  100. ,  125. ,
        150. ,  175. ,  200. ,  250. ,  300. ,  550. ,  800. , 1050. ,
       1300. ]), name='mreco', label='m_{RECO} (GeV)'),
  storage=Weight()) # Sum: WeightedSum(value=7.72514e+08, variance=1.01009e+13) (WeightedSum(value=7.73556e+08, variance=1.01117e+13) with flow)
Hist(
  StrCategory(['herwig2016APV', 'herwig2016', 'herwig2017', 'herwig2018'], growth=True, name='dataset', label='Primary dataset'),
  IntCategory([-1], growth=True, name='jk', label='Jackknife section'),
  Variable([200, 290, 400, 480, 570, 680, 760, 820, 13000], name='ptgen', label='p_{T,GEN} (GeV)'),
  Varia

In [3]:
from python.plugins import addFiles
# files = ["coffeaOutput/dijet/dijetHists_fixherwigxs_QCDsim_rap2.5_MG_nominal_JK0_2_2016APV.pkl",
#          "coffeaOutput/dijet/dijetHists_fixherwigxs_QCDsim_rap2.5_MG_nominal_JK2_4_2016APV.pkl", 
#          "coffeaOutput/dijet/dijetHists_fixherwigxs_QCDsim_rap2.5_MG_nominal_JK4_6_2016APV.pkl", 
#          "coffeaOutput/dijet/dijetHists_fixherwigxs_QCDsim_rap2.5_MG_nominal_JK6_8_2016APV.pkl", 
#          "coffeaOutput/dijet/dijetHists_fixherwigxs_QCDsim_rap2.5_MG_nominal_JK8_10_2016APV.pkl" ]
# combinedFile = addFiles(files)
# if "ALL" in files[0]:
#     year = "Run2UL"
# elif "APV" in files[0]:
#     year = "2016APV"
# else:
#     year = files[0][-8:-4]
# print(combinedFile)
# with open("coffeaOutput/dijet/dijetHists_fixherwigxs_QCDsim_rap2.5_MG_nominal_JK"+year+".pkl", "wb") as f:
#     pickle.dump( combinedFile, f)
# fname = "coffeaOutput/dijet/dijetHists_fixherwigxs_QCDsim_rap2.5_MG_nominal_JK"+year+".pkl"
# with open(fname, "rb") as f:
#     result = pickle.load( f )
# availSysts = [ax for ax in result['ptreco_mreco_u'].project("syst").axes[0]]

In [4]:
def combineFiles(prepstr, IOV="ALL", RespOnly=False, inds=None):
    unc_srcs = ["nominal","HEM", "JER", "JMR", "JMS", "AbsoluteMPFBias","AbsoluteScale","AbsoluteStat","FlavorQCD","Fragmentation","PileUpDataMC","PileUpPtBB","PileUpPtEC1",
"PileUpPtEC2","PileUpPtHF","PileUpPtRef","RelativeFSR","RelativeJEREC1",
            "RelativeJEREC2","RelativeJERHF","RelativePtBB","RelativePtEC1",
            "RelativePtEC2","RelativePtHF","RelativeBal","RelativeSample",
            "RelativeStatEC","RelativeStatFSR","RelativeStatHF","SinglePionECAL",
            "SinglePionHCAL","TimePtEta"]
    iovs = ["2016APV", "2016", "2017", "2018"]
    # unc_srcs = ["nominal","JER","JMR","JMS","HEM","AbsoluteMPFBias","AbsoluteScale","AbsoluteStat","FlavorQCD","Fragmentation","PileUpDataMC","PileUpPtBB","PileUpPtEC1","PileUpPtEC2"]
    files = []
    if IOV == "ALL" and inds==None:
        print("Adding years or not systs")
        for i in iovs:
            files.append(prepstr+i+".pkl")
    elif IOV == "ALL" and inds!=None:
        print("Adding sections")
        for i in iovs:
            for ind in inds:
                files.append(prepstr+i+".pkl")
    else:
        for src in unc_srcs:
            files.append(prepstr+src+IOV+".pkl")
    print("Final list of files ", files)
    combined = addFiles(files, RespOnly=RespOnly)
    outputFilename = prepstr+"allUnc"+IOV+".pkl"
    if RespOnly: outputFilename = prepstr+"allUnc"+IOV+"RespOnly.pkl"
    with open(outputFilename, "wb") as f:
        pickle.dump( combined, f)
    return combined
def combineSystFiles(prepstr, IOV, RespOnly=False):
    unc_srcs = ["nominal","HEM", "JER", "JMR", "JMS", "AbsoluteMPFBias","AbsoluteScale","AbsoluteStat","FlavorQCD","Fragmentation","PileUpDataMC","PileUpPtBB","PileUpPtEC1",
"PileUpPtEC2","PileUpPtHF","PileUpPtRef","RelativeFSR","RelativeJEREC1",
            "RelativeJEREC2","RelativeJERHF","RelativePtBB","RelativePtEC1",
            "RelativePtEC2","RelativePtHF","RelativeBal","RelativeSample",
            "RelativeStatEC","RelativeStatFSR","RelativeStatHF","SinglePionECAL",
            "SinglePionHCAL","TimePtEta"]
    iovs = ["2016APV", "2016", "2017", "2018"]
    # unc_srcs = ["nominal","JER","JMR","JMS","HEM","AbsoluteMPFBias","AbsoluteScale","AbsoluteStat","FlavorQCD","Fragmentation","PileUpDataMC","PileUpPtBB","PileUpPtEC1","PileUpPtEC2"]
    files = []
    if IOV == "ALL":
        print("Adding years not systs")
        for i in iovs:
            files.append(prepstr+i+".pkl")
    else:
        for src in unc_srcs:
            files.append(prepstr+src+IOV+".pkl")
    print("Final list of files ", files)
    combined = addFiles(files, RespOnly=RespOnly)
    print("Combined file hists", combined.keys())
    outputFilename = prepstr+"allUnc"+IOV+".pkl"
    print("output filename ", outputFilename)
    if RespOnly: outputFilename = prepstr+"allUnc"+IOV+"RespOnly.pkl"
    with open(outputFilename, "wb") as f:
        pickle.dump( combined, f)
    return combined

In [31]:
# combineSystFiles("coffeaOutput/trijet/trijetHistsTest_wXSscaling_QCDsim_pt200.0_rapidity2.5_", "2018")

#combineSystFiles("coffeaOutput/dijet/dijetHists_wXSscaling_QCDsim_pt200.0_rapidity2.5_", "2018", RespOnly=False)
combineSystFiles("coffeaOutput/dijet/dijetHists_fixSDmass_QCDsim_rap2.5_herwig_nominal__", "ALL", RespOnly=True)



Adding years not systs
Final list of files  ['coffeaOutput/dijet/dijetHists_fixSDmass_QCDsim_rap2.5_herwig_nominal__2016APV.pkl', 'coffeaOutput/dijet/dijetHists_fixSDmass_QCDsim_rap2.5_herwig_nominal__2016.pkl', 'coffeaOutput/dijet/dijetHists_fixSDmass_QCDsim_rap2.5_herwig_nominal__2017.pkl', 'coffeaOutput/dijet/dijetHists_fixSDmass_QCDsim_rap2.5_herwig_nominal__2018.pkl']
dict_keys(['misses_u', 'misses_g', 'fakes_u', 'fakes_g', 'underflow', 'underflow_g', 'weights', 'ptreco_mreco_u', 'ptreco_mreco_g', 'rho_reco_u', 'rho_reco_g', 'ptgen_mgen_u', 'ptgen_mgen_g', 'response_rho_u', 'response_rho_g', 'response_matrix_u', 'response_matrix_g', 'cutflow', 'jkflow', 'mass_orig', 'sdmass_orig', 'sdmass_ak8corr', 'sdmass_ak4corr', 'njet_reco', 'njet_gen', 'jet_rap_reco', 'jet_rap_gen', 'jet_phi_gen', 'jet_phi_reco', 'jet_eta_phi_precuts', 'jet_eta_phi_preveto', 'jet_pt_eta_phi', 'dphi_gen', 'dphi_reco', 'asymm_gen', 'asymm_reco', 'jet_dr_gen_subjet', 'dijet_dr_reco_to_gen', 'dr_reco_to_gen_subje

{'response_matrix_u': Hist(
   StrCategory(['herwig2016APV', 'herwig2016', 'herwig2017', 'herwig2018'], growth=True, name='dataset', label='Primary dataset'),
   StrCategory(['nominal', 'L1prefiringDown', 'PUSFUp', 'PDFDown', 'PUSFDown', 'LuminosityUp', 'L1prefiringUp', 'LuminosityDown', 'Q2Down', 'PDFUp', 'Q2Up'], growth=True, name='syst', label='Systematic'),
   IntCategory([-1], growth=True, name='jk', label='Jackknife section'),
   Variable([200, 290, 400, 480, 570, 680, 760, 13000], name='ptreco', label='$p_{T,RECO}$ [GeV]'),
   Variable(array([   0. ,    2.5,    5. ,    7.5,   10. ,   15. ,   20. ,   30. ,
          40. ,   50. ,   60. ,   70. ,   80. ,   90. ,  100. ,  125. ,
         150. ,  175. ,  200. ,  250. ,  300. ,  350. ,  400. ,  450. ,
         500. ,  700. ,  900. , 1100. , 1300. ]), name='mreco', label='$m_{RECO}$ [GeV]'),
   Variable([200, 290, 400, 480, 570, 680, 760, 13000], name='ptgen', label='$p_{T,GEN}$ [GeV]'),
   Variable([0, 5, 10, 20, 40, 60, 80, 100, 150

In [6]:
fname = "coffeaOutput/trijetHists_wXSscaling_QCDsim_pt200.0rapidity2.5_bloosejesjecALL.pkl"
with open(fname, "rb") as f:
    result = pickle.load( f )
print("values: ", result['response_matrix_u'].project("ptgen", "mgen").values())
print("variances: ", result['response_matrix_u'].project("ptgen", "mgen").variances())

values:  [[0.00000000e+00 1.64840196e-01 1.18401463e+02 7.38814285e+03
  1.20690132e+05 1.64512215e+05 7.51661600e+04 2.98801216e+04
  9.88462497e+03 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 4.83658556e-01 6.37905239e+00 4.69203085e+02
  1.04213764e+04 2.27263886e+04 1.60053940e+04 7.47822871e+03
  7.56164125e+03 1.34039913e+02 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 2.38251690e+00 5.47565308e+01
  1.01231259e+03 3.37410983e+03 2.72137159e+03 1.88611583e+03
  1.91757304e+03 4.36158293e+02 9.99400201e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 4.05365298e-03 5.08970139e+00
  9.72279846e+01 3.71238657e+02 3.95851684e+02 2.78597575e+02
  3.30672076e+02 1.42670520e+02 1.83631184e+01 1.38024987e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 1.30384677e-01
  2.02193621e+01 1.02960264e+02 1.35803815e+02 9.55402662e+01
  1.18882460e+02 4.99535960e+01 2.74412044e+01 5.02359511e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 1.2651172

In [7]:
%matplotlib inline
# fname = "coffeaOutput/dijetHists_wXSscaling_QCDsim_pt200.0_rapidity2.5jesjecALL.pkl"
#fname = "coffeaOutput/trijetHists_wXSscaling_QCDsim_pt200.0rapidity2.5_bloosejesjecALL.pkl" <-- has reco hist bug
fname = "coffeaOutput/dijetHists_wXSscaling_QCDsim_pt200.0_rapidity2.5jesjecL1PU2016.pkl"
# fname ="coffeaOutput/trijetHistsTest_JetHT_pt200.0_eta2.4_bbloose.pkl"
eras = ["2016", "2017", "2018", "2016APV"]
dir = "rootFiles/"
for year in eras:
    print(year)
    if year in fname:
        print(year)
        year_str = year
        break
    else:
        year_str = "ALL"
with open(fname, "rb") as f:
    result = pickle.load( f )
print([ax for ax in result['jet_pt_mass_reco_u'][{'dataset':sum}].axes[0]])
axis_names = [ax for ax in result['jet_pt_mass_reco_u'][{'dataset':sum}].axes[0]]
print(axis_names)
cats = [cat for cat in result['jet_pt_mass_reco_u'][{'ptreco':sum, 'dataset':sum, 'mreco':sum}].axes[0]]
print(cats)
if "JetHT" in fname:
    if "dijet" in  fname:
        rootfname = 'dijetHistsJetHT_jec_' + year_str + '.root'
    elif "trijet" in fname:
        rootfname = 'trijetHistsJetHT_jec_' + year_str + '.root'
elif "QCD" in fname:
    if "dijet" in  fname:
        rootfname = 'dijetHistsQCDsim_jec_' + year_str + '.root'
    # integrate and sum over axes
    elif "trijet" in fname:
        rootfname = 'trijetHistsQCDsim_jec' + year_str + '.root'
if os.path.exists(dir + rootfname):
    rootfile = uproot.recreate(dir+rootfname)
else:
    rootfile = uproot.create(dir+rootfname)
    #### only add hists containing gen if SIM files
response_entries = {}
ptgen_mgen_entries = {}
ptreco_mreco_entries = {}
fakes_entries = {}
misses_entries = {}
if "QCD" in fname:
    for syst in cats:
        rootfile['ptgen_mgen_u_'+syst] = result['response_matrix_u'][{'syst':syst}].project("ptgen", "mgen")
        rootfile['ptgen_mgen_g_'+syst] = result['response_matrix_g'][{'syst':syst}].project("ptgen","mgen")
        rootfile['ptreco_mreco_u_'+syst] = result['response_matrix_u'][{'syst':syst}].project("ptreco", "mreco")
        rootfile['ptreco_mreco_g_'+syst] = result['response_matrix_g'][{'syst':syst}].project("ptreco","mreco")
        rootfile['misses_ptgen_mgen_'+syst] = result['misses'][{'syst':syst}].project("ptgen", "mgen")
        rootfile['fakes_ptreco_mreco_'+syst] = result['fakes'][{'syst':syst}].project("ptreco","mreco")
        response_matrix_u_values, ptreco_edges, mreco_edges, ptgen_edges, mgen_edges = result['response_matrix_u'][{'syst':syst}].project("ptreco", "mreco", "ptgen", "mgen").to_numpy(flow=True)
        response_matrix_g_values, ptreco_edges, mreco_edges, ptgen_edges, mgen_edges= result['response_matrix_g'][{'syst':syst}].project("ptreco", "mreco", "ptgen", "mgen").to_numpy(flow=True)
        print(response_matrix_g_values.shape)
        ptreco_centers = (ptreco_edges[:-1]+ptreco_edges[1:])/2
        print(ptreco_centers[np.newaxis])
        mreco_centers = (mreco_edges[:-1]+mreco_edges[1:])/2
        ptgen_centers = (ptgen_edges[:-1]+ptgen_edges[1:])/2
        mgen_centers = (mgen_edges[:-1]+mgen_edges[1:])/2
        print(np.shape(response_matrix_g_values))
        print(np.shape(response_matrix_g_values[np.newaxis]))
        response_entries.update({'groomed_'+syst:response_matrix_g_values[np.newaxis],
                           'ungroomed_'+syst:response_matrix_u_values[np.newaxis]})
    rootfile['response']=response_entries
else:
    for syst in cats:
        rootfile['ptreco_mreco_u_'+syst] = result['jet_pt_mass_reco_u'][{'syst':syst}].project("ptreco", "mreco")
        rootfile['ptreco_mreco_g_'+syst] = result['jet_pt_mass_reco_g'][{'syst':syst}].project("ptreco","mreco")
rootfile.close()

2016
2016
['nominal', 'jesUp', 'jesDown', 'jerUp', 'jerDown']
['nominal', 'jesUp', 'jesDown', 'jerUp', 'jerDown']
['nominal', 'jesUp', 'jesDown', 'jerUp', 'jerDown']
(12, 26, 12, 14)
[[ -inf  240.  320.  405.  485.  575.  660.  720.  775. 1050. 7150.   inf]]
(12, 26, 12, 14)
(1, 12, 26, 12, 14)
(12, 26, 12, 14)
[[ -inf  240.  320.  405.  485.  575.  660.  720.  775. 1050. 7150.   inf]]
(12, 26, 12, 14)
(1, 12, 26, 12, 14)
(12, 26, 12, 14)
[[ -inf  240.  320.  405.  485.  575.  660.  720.  775. 1050. 7150.   inf]]
(12, 26, 12, 14)
(1, 12, 26, 12, 14)
(12, 26, 12, 14)
[[ -inf  240.  320.  405.  485.  575.  660.  720.  775. 1050. 7150.   inf]]
(12, 26, 12, 14)
(1, 12, 26, 12, 14)
(12, 26, 12, 14)
[[ -inf  240.  320.  405.  485.  575.  660.  720.  775. 1050. 7150.   inf]]
(12, 26, 12, 14)
(1, 12, 26, 12, 14)


In [8]:
import uproot
fname = "coffeaOutput/dijetHists_wXSscaling_QCDsim_pt200.0_rapidity2.5jesjecL1PU2016.pkl"
with open(fname, "rb") as f:
    result = pickle.load( f )
print(result.keys())
response_matrix_u_values, ptreco_edges, mreco_edges, ptgen_edges, mgen_edges = result['response_matrix_u'][{'syst':syst}].project("ptreco", "mreco", "ptgen", "mgen").to_numpy(flow=True)
print(response_matrix_g_values.shape)
response_matrix_g_values= result['response_matrix_g'][{'syst':'nominal'}].project("ptreco", "mreco", "ptgen", "mgen").values()
nptreco,nmassreco,nptgen,nmassgen = response_matrix_g_values.shape
response_matrix_g_final = response_matrix_g_values.reshape( (nptreco)*(nmassreco), (nptgen)*(nmassgen) ) 
response_matrix_u_final = response_matrix_u_values.reshape( (nptreco+2)*(nmassreco+2), (nptgen+2)*(nmassgen+2) ) 
print(response_matrix_g_final.shape)
response_matrix_g_final2 = result['response_matrix_g'][{'syst':"nominal"}].project("ptreco", "mreco", "ptgen", "mgen").values(flow=True).reshape( (nptreco+2)*(nmassreco+2), (nptgen+2)*(nmassgen+2))
ptreco_mreco_values= result['jet_pt_mass_reco_g'][{'syst':'nominal'}].project("ptreco", "mreco").values(flow=True)
print(ptreco_mreco_values.shape)
print(ptreco_mreco_values[1][0])
print(np.sum(response_matrix_g_final[24,:]))
# print(np.sum(response_matrix_g_final2[26,:]))
# print(np.sum(response_matrix_u_final[26,:]))

dict_keys(['jet_mass', 'jet_pt', 'jet_rap', 'jet_eta', 'njet_gen', 'jet_mass_u_reco_over_gen', 'jet_pt_gen', 'jet_mass_gen', 'jet_pt_reco_over_gen', 'jet_dphi_gen', 'jet_ptasymm_gen', 'jet_dr_gen_subjet', 'dijet_dr_reco_to_gen', 'dr_reco_to_gen_subjet', 'misses', 'fakes', 'weights', 'jet_pt_mass_reco_u', 'jet_pt_mass_reco_g', 'jet_pt_mass_gen_u', 'jet_pt_mass_gen_g', 'jet_m_pt_u_reco_over_gen', 'jet_m_pt_g_reco_over_gen', 'response_matrix_u', 'response_matrix_g', 'cutflow'])
(12, 26, 12, 14)
(240, 120)
(12, 26)
36867894.01413605
6684116.343194973


In [9]:
# fname = uproot.open('root://cmsxrootd.fnal.gov//store/mc/RunIISummer20UL18NanoAODv9/QCD_Pt_1400to1800_TuneCP5_13TeV_pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/280000/42B4DEA0-C0E7-A944-80DC-1D107A0F63FB.root')

In [10]:
# print(fname["Events"].keys("*HTXS*"))

In [11]:
hists = uproot.open('rootFiles/dijetHistsQCDsim_jec_2016.root')
print(hists.keys())
hist_vals_g, hist_edges, hist_widths = hists['ptreco_mreco_g_nominal'].to_numpy(flow=False)
hist_vals_u, hist_edges, hist_widths = hists['ptreco_mreco_u_nominal'].to_numpy(flow=False)
print(hist_vals_u.shape)
print(hist_vals_g[1][0])
print(hist_vals_u[1][0])

['ptgen_mgen_u_nominal;1', 'ptgen_mgen_g_nominal;1', 'ptreco_mreco_u_nominal;1', 'ptreco_mreco_g_nominal;1', 'misses_ptgen_mgen_nominal;1', 'fakes_ptreco_mreco_nominal;1', 'ptgen_mgen_u_jesUp;1', 'ptgen_mgen_g_jesUp;1', 'ptreco_mreco_u_jesUp;1', 'ptreco_mreco_g_jesUp;1', 'misses_ptgen_mgen_jesUp;1', 'fakes_ptreco_mreco_jesUp;1', 'ptgen_mgen_u_jesDown;1', 'ptgen_mgen_g_jesDown;1', 'ptreco_mreco_u_jesDown;1', 'ptreco_mreco_g_jesDown;1', 'misses_ptgen_mgen_jesDown;1', 'fakes_ptreco_mreco_jesDown;1', 'ptgen_mgen_u_jerUp;1', 'ptgen_mgen_g_jerUp;1', 'ptreco_mreco_u_jerUp;1', 'ptreco_mreco_g_jerUp;1', 'misses_ptgen_mgen_jerUp;1', 'fakes_ptreco_mreco_jerUp;1', 'ptgen_mgen_u_jerDown;1', 'ptgen_mgen_g_jerDown;1', 'ptreco_mreco_u_jerDown;1', 'ptreco_mreco_g_jerDown;1', 'misses_ptgen_mgen_jerDown;1', 'fakes_ptreco_mreco_jerDown;1', 'response;1']
(10, 24)
6696251.456774509
4889.022838239164


In [12]:
import uproot
hists = uproot.open(rootFiles/dijetHistsQCDsim_jec_2016.root)
print(hists['response'].keys())
resp_groomed = hists['response']['groomed_nominal'].array().to_numpy()[0].reshape((nptreco)*(nmassreco), (nptgen)*(nmassgen))
print(np.sum(resp_groomed[24,:]))
plt.figure(figsize = (20,20))
plt.imshow( resp_groomed, vmax=10, aspect="equal", cmap="Blues" )
plt.xlabel("GEN", fontsize=10)
plt.ylabel("RECO", fontsize=10)
plt.tick_params(labelsize=20)

NameError: name 'rootFiles' is not defined